In [6]:
from dash import Dash, html, dcc, callback, Input, Output, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash_bootstrap_templates import load_figure_template

In [104]:
# Load the data
athlete_events = pd.read_csv('Data/athlete_events.csv')
athlete_events.sort_values(by='Year', inplace=True)
athlete_events.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
214333,107607,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,1896,Summer,Athina,Gymnastics,Gymnastics Men's Pommelled Horse,NaN
244717,122526,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,1896,Summer,Athina,Athletics,Athletics Men's Triple Jump,Silver
244716,122526,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,1896,Summer,Athina,Athletics,Athletics Men's Long Jump,NaN
23912,12563,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,1896,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN
23913,12563,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,1896,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN


In [145]:
load_figure_template("flatly")

app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('Olympic Games', style={'fontSize': '30px'}, className='mt-3 text-left text-primary font-weight-bold'),
            html.H3('Year', style={'fontSize': '24px'}, className="mt-2"),
            dcc.Dropdown(
                id='year_dropdown',
                options=[{'label': i, 'value': i} for i in athlete_events['Year'].unique()],
                value=None,
                style={'width': '50%'}
            ),
            html.H3('Country', style={'fontSize': '24px'}, className="mt-2"),
            dcc.Dropdown(
                id='country_dropdown',
                options=[{'label': i, 'value': i} for i in athlete_events['NOC'].unique()],
                value=None,
                style={'width': '50%'}
            )
        ], width=4),

        dbc.Col([
            html.H3('No. of competitors', style={'fontSize': '24px'}),
            html.H3(id="no_competitors")
        ], width=2),

        dbc.Col([
            html.H3('No. of Medals', style={'fontSize': '24px'}),
            html.H3(id="no_medals")
        ], width=2),

        dbc.Col([
            html.H3('Medal Breakdown By Top 10 Sports & Type', style={'fontSize': '16px'}),
            dcc.Graph(id='medal_breakdown_sport')
        ], width=3),
    ], justify='left'),
    dbc.Row([
        dbc.Col([
            # html.H3('Country', style={'fontSize': '24px'}),
            # dcc.Dropdown(
            #     id='country_dropdown',
            #     options=[{'label': i, 'value': i} for i in athlete_events['NOC'].unique()],
            #     value=None
            # )
        ], width=3),
    ]),
], fluid=True)

@callback(
    Output('no_competitors', 'children'),
    Output('no_medals', 'children'),
    Output('medal_breakdown_sport', 'figure'),
    Input('year_dropdown', 'value'),
    Input('country_dropdown', 'value')  # Add the country dropdown input
)

def update_graph(year, country):
    if year is None and country is None:
        no_of_competitors = athlete_events['Name'].nunique()  # Sum of all years and countries combined
    elif year is None:
        no_of_competitors = athlete_events[athlete_events['NOC'] == country]['Name'].nunique()
    elif country is None:
        no_of_competitors = athlete_events.groupby('Year').get_group(year)['Name'].nunique()
    else:
        no_of_competitors = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)]['Name'].nunique()

    if year is None and country is None:
        no_of_medals = athlete_events['Medal'].count()
    elif year is None:
        no_of_medals = athlete_events[athlete_events['NOC'] == country]['Medal'].count()
    elif country is None:
        no_of_medals = athlete_events.groupby('Year').get_group(year)['Medal'].count()
    else:
        no_of_medals = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)]['Medal'].count()
    
    if year is None and country is None:
        medal_breakdown = athlete_events.groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig.update_traces(marker_color='gold', selector=dict(name='Gold'))
    elif year is None:
        medal_breakdown = athlete_events[athlete_events['NOC'] == country].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig.update_traces(marker_color='gold', selector=dict(name='Gold'))
    elif country is None:
        medal_breakdown = athlete_events[athlete_events['Year'] == year].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig.update_traces(marker_color='gold', selector=dict(name='Gold'))
    else:
        medal_breakdown = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        medal_breakdown = medal_breakdown.groupby(['Sport', 'Medal'])['Count'].sum().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig.update_traces(marker_color='gold', selector=dict(name='Gold'))
    

    
    return no_of_competitors, no_of_medals, fig

app.run(debug=True, jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/


/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_57370/3515907308.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_57370/3515907308.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

